--------------------------------------------------------------------------------

# Created by [Nikolaos G. Giakoumoglou](https://www.iti.gr/iti/people/Nikolaos_Giakoumoglou.html)

### Electrical and Computer Engineer

Mobile: +30 6906430995 | e-mail: [ngiakoumoglou@gmail.gr](mailto:ngiakoumoglou@gmail.gr) | e-mail: [ngiakoumoglou@hotmail.gr](mailto:ngiakoumoglou@hotmail.gr) |

-------------------------------------------------------------------------------

# YOLOv5

This tutorial is based on the [YOLOv5 repository](https://github.com/ultralytics/yolov5) by [Ultralytics](https://www.ultralytics.com/). 

# 1 Install Dependencies

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt 

!pip install pycocotools
!pip install onnx>=1.9.0
!pip install roboflow

import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# 2 Download Dataset 

We'll download our dataset from Roboflow. Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.


In [ ]:
%%capture
from roboflow import Roboflow
rf = Roboflow(api_key="7EOGxE0Zhj6CaawbEz1J")
project = rf.workspace("joseph-nelson").project("mask-wearing")
dataset = project.version(4).download("yolov5")

## 2.1 Create custom yaml files for each model

In [ ]:
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
!cp -r ./models/yolov5n.yaml ./models/custom_yolov5n.yaml
!cp -r ./models/yolov5s.yaml ./models/custom_yolov5s.yaml
!cp -r ./models/yolov5m.yaml ./models/custom_yolov5m.yaml
!cp -r ./models/yolov5l.yaml ./models/custom_yolov5l.yaml
!cp -r ./models/yolov5x.yaml ./models/custom_yolov5x.yaml

%cd models
!sed -i 's/nc: 80/nc: {num_classes}/' custom_yolov5n.yaml
!sed -i 's/nc: 80/nc: {num_classes}/' custom_yolov5s.yaml
!sed -i 's/nc: 80/nc: {num_classes}/' custom_yolov5m.yaml
!sed -i 's/nc: 80/nc: {num_classes}/' custom_yolov5l.yaml
!sed -i 's/nc: 80/nc: {num_classes}/' custom_yolov5x.yaml
%cd ..

# 3 Train Custom YOLOv5 Detector

To kick off training we running the training command with the following options:

* _img_: define input image size
* _batch_: determine batch size
* _epochs_: define the number of training epochs.
* _data_: set the path to our yaml file
* _cfg_: specify our model configuration
* _weights_: specify a custom path to weights. (Note: you can * download weights from the Ultralytics Google Drive folder)
* _name_: result names
* _nosave_: only save the final checkpoint
* _cache_: cache images for faster training
* _hyps_: hyperparameters of training



In [ ]:
img =   1024#@param {type:"integer"}
batch =  4#@param {type:"integer"}
epochs = 300 #@param {type:"integer"}
cfg = 'models/custom_yolov5n.yaml' #@param ["models/custom_yolov5n.yaml", "models/custom_yolov5s.yaml", "models/custom_yolov5m.yaml", "models/custom_yolov5l.yaml", "models/custom_yolov5x.yaml"]
weights = 'yolov5n.pt' #@param ["yolov5n.pt", "yolov5s.pt", "yolov5m.pt", "yolov5l.pt", "yolov5x.pt"]
optimizer = 'SGD' #@param ["SGD", "Adam", "AdamW"]
hyp = 'data/hyps/hyp.scratch-low.yaml' #@param["data/hyps/hyp.scratch-low.yaml", "data/hyps/hyp.scratch-med.yaml", "data/hyps/hyp.scratch-high.yaml"]
name = 'experiment-1' #@param {type:"string"}

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img {img} --batch {batch} --epochs {epochs} --data {dataset.location}/data.yaml --cfg {cfg} --weights {weights} --optimizer {optimizer} --hyp {hyp} --name {name} --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 4 Validate Custom YOLOv5 Detector 



In [ ]:
!python val.py --weights runs/train/{name}}/weights/best.pt --img {img} --data {dataset.location}/data.yaml --name {name} --task val